In [1]:
#导入相关数据库
from KRData.HKData import HKFuture
import talib as ta
import pandas as pd
import numpy as np

In [2]:
#导入相关数据库
from KRData.HKData import HKFuture
import talib
import pandas as pd
import numpy as np

In [3]:
import datetime as dt

In [4]:
#连接数据库
hf = HKFuture('', '')

In [5]:
#取数据，由20110101-20190401之间HSI的数据
trade_dates=hf.get_main_contract_trade_dates('HSI','20110101','20190401')

In [6]:
ret_list=[]
ret_list2=[]
#设置一个循环来遍历每个合约的代码和对应的交易时间段
for code,tds in trade_dates.items():
    #输出结果
    print(f'code:{code}    {tds[0]}->{tds[-1]}')
    data=hf.get_bars(code,fields=['datetime','open','high','low','close','trade_date'],start=tds[0],end=tds[-1])
    data['last_close'] = data['close'].shift(1)
    #计算MA60
    data['ma60']=talib.MA(data['close'],timeperiod=60)
    data['gt_ma60']=data['close']>data['ma60']
    data['gt_ma60_1']=data['gt_ma60'].shift(1).fillna(0)
    #通过位运算确定金叉死叉情况，1为金叉，2为死叉
    data['ma60_type'] = (data['gt_ma60_1'].values << 1) + data['gt_ma60'].values
    #计算MACD
    data['macd'],data['diff'],data['dea']=ta.MACD(data.close.values,fastperiod=6,slowperiod=12,signalperiod=9)
    data['macd_gt0'] = (data['macd']>=0).astype(np.int8)
    data['macd_gt0_1'] = (data['macd_gt0'].shift(1).fillna(0)).astype(np.int8)
    #通过位运算计算红区和绿区
    data['macd_type'] = (data['macd_gt0_1'].values << 1) + data['macd_gt0'].values
    #取出MA60金叉死叉的两种情况
    ma60_type = data[(data['ma60_type']==1)|(data['ma60_type']==2)]
    #对应的时间做一个shift位移求用于之后求对应金叉死叉区间
    ma60_type['datetime_1'] = ma60_type['datetime'].shift(-1)
    #金叉情况
    d=ma60_type[ma60_type['ma60_type']==1].loc[:, ['datetime', 'datetime_1']]
    #这里增加了一个过滤条件，区间持续20分钟以上的才算一个
    for i, r in d[(d['datetime_1'] - d['datetime']) > dt.timedelta(minutes=20)].iterrows():
        ma60_df = data[r.datetime:r.datetime_1]
        macd_type = ma60_df[(ma60_df['macd_type']==1)|(ma60_df['macd_type']==2)]
        macd_type['macd_close'] = macd_type['close'].shift(-1)
        macd_type['macd_section_pnl'] = macd_type['macd_close'] - macd_type['close']
        ret = macd_type[macd_type['macd_type'] == 1].loc[:, ['macd_section_pnl']]
        ret['count'] = range(len(ret))
        ret_list.append(ret)

        
    #死叉情况
    d=ma60_type[ma60_type['ma60_type']==2].loc[:, ['datetime', 'datetime_1']]
    for i, r in d[(d['datetime_1'] - d['datetime']) > dt.timedelta(minutes=20)].iterrows():
        ma60_df = data[r.datetime:r.datetime_1]
        macd_type = ma60_df[(ma60_df['macd_type']==1)|(ma60_df['macd_type']==2)]
        macd_type['macd_close'] = macd_type['close'].shift(-1)
        macd_type['macd_section_pnl'] = macd_type['macd_close'] - macd_type['close']
        ret = macd_type[macd_type['macd_type'] == 2].loc[:, ['macd_section_pnl']]
        ret['count'] = range(len(ret))
        ret_list2.append(ret)
    

code:HSI1101    2011-01-03 00:00:00->2011-01-28 00:00:00


D:\Programs\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Programs\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Programs\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

code:HSI1102    2011-01-31 00:00:00->2011-02-25 00:00:00
code:HSI1103    2011-02-28 00:00:00->2011-03-30 00:00:00
code:HSI1104    2011-03-31 00:00:00->2011-04-28 00:00:00
code:HSI1105    2011-04-29 00:00:00->2011-05-30 00:00:00
code:HSI1106    2011-05-31 00:00:00->2011-06-29 00:00:00
code:HSI1107    2011-06-30 00:00:00->2011-07-28 00:00:00
code:HSI1108    2011-07-29 00:00:00->2011-08-30 00:00:00
code:HSI1109    2011-08-31 00:00:00->2011-09-30 00:00:00
code:HSI1110    2011-10-03 00:00:00->2011-10-28 00:00:00
code:HSI1111    2011-10-31 00:00:00->2011-11-29 00:00:00
code:HSI1112    2011-11-30 00:00:00->2011-12-29 00:00:00
code:HSI1201    2011-12-30 00:00:00->2012-01-30 00:00:00
code:HSI1202    2012-01-31 00:00:00->2012-02-28 00:00:00
code:HSI1203    2012-02-29 00:00:00->2012-03-29 00:00:00
code:HSI1204    2012-03-30 00:00:00->2012-04-27 00:00:00
code:HSI1205    2012-04-30 00:00:00->2012-05-30 00:00:00
code:HSI1206    2012-05-31 00:00:00->2012-06-28 00:00:00
code:HSI1207    2012-06-29 00:0

In [15]:
total_ret1 = pd.concat([ ret for ret in ret_list])

In [13]:
total_ret2 = pd.concat([ ret for ret in ret_list2])

In [ ]:
#金叉统计情况

In [22]:
print(total_ret1.groupby('count')['macd_section_pnl'].describe())

        count        mean        std    min     25%    50%     75%     max
count                                                                     
0.0    7669.0   14.152693  50.095082 -337.0   -7.00    2.0   23.00  1018.0
1.0    4250.0    0.457882  36.092325 -451.0  -12.00   -5.0    2.00   366.0
2.0    2106.0    0.783476  37.011453 -320.0  -12.00   -5.0    1.00   513.0
3.0     938.0    2.320896  45.134856 -307.0  -11.00   -5.0    2.00   559.0
4.0     382.0    0.633508  31.579533 -135.0  -12.00   -5.0    1.00   239.0
5.0     149.0   -1.181208  27.461780 -148.0  -10.00   -5.0    1.00   127.0
6.0      61.0   -0.163934  30.485068  -60.0  -10.00   -5.0    3.00   130.0
7.0      27.0    6.444444  31.653574  -20.0  -11.00   -6.0    9.50   111.0
8.0      14.0   -3.642857  15.494948  -21.0  -10.25   -5.5   -2.25    45.0
9.0       4.0  -11.250000   6.652067  -18.0  -15.75  -12.0   -7.50    -3.0
10.0      1.0  121.000000        NaN  121.0  121.00  121.0  121.00   121.0
11.0      1.0  -12.000000

In [19]:
print(total_ret1.groupby('count')['macd_section_pnl'].aggregate(['sum','mean','max','min','std']))

            sum        mean     max    min        std
count                                                
0.0    108537.0   14.152693  1018.0 -337.0  50.095082
1.0      1946.0    0.457882   366.0 -451.0  36.092325
2.0      1650.0    0.783476   513.0 -320.0  37.011453
3.0      2177.0    2.320896   559.0 -307.0  45.134856
4.0       242.0    0.633508   239.0 -135.0  31.579533
5.0      -176.0   -1.181208   127.0 -148.0  27.461780
6.0       -10.0   -0.163934   130.0  -60.0  30.485068
7.0       174.0    6.444444   111.0  -20.0  31.653574
8.0       -51.0   -3.642857    45.0  -21.0  15.494948
9.0       -45.0  -11.250000    -3.0  -18.0   6.652067
10.0      121.0  121.000000   121.0  121.0        NaN
11.0      -12.0  -12.000000   -12.0  -12.0        NaN
12.0       20.0   20.000000    20.0   20.0        NaN
13.0      -20.0  -20.000000   -20.0  -20.0        NaN


In [ ]:
#死叉统计情况

In [20]:
print(total_ret2.groupby('count')['macd_section_pnl'].describe())

        count       mean        std    min    25%   50%    75%    max
count                                                                
0.0    7289.0 -15.105090  57.010653 -775.0 -22.00  -2.0   8.00  594.0
1.0    3966.0   0.123550  39.515494 -501.0  -1.00   6.0  13.00  453.0
2.0    1823.0   0.744377  38.826480 -465.0  -1.00   6.0  13.00  521.0
3.0     765.0  -0.881046  31.945064 -215.0  -1.00   6.0  13.00   68.0
4.0     297.0  -1.629630  35.863257 -322.0  -2.00   6.0  13.00  149.0
5.0     121.0  -4.528926  37.895267 -236.0  -5.00   6.0  11.00   54.0
6.0      55.0  -2.436364  38.421523 -228.0 -13.00   6.0  15.50   36.0
7.0      20.0   0.800000  47.672788 -131.0  -2.25   6.0  11.00  116.0
8.0       4.0  11.500000  12.124356   -3.0   3.75  13.5  21.25   22.0
9.0       2.0 -36.500000  16.263456  -48.0 -42.25 -36.5 -30.75  -25.0
10.0      1.0 -29.000000        NaN  -29.0 -29.00 -29.0 -29.00  -29.0
11.0      1.0 -13.000000        NaN  -13.0 -13.00 -13.0 -13.00  -13.0


In [21]:
print(total_ret2.groupby('count')['macd_section_pnl'].aggregate(['sum','mean','max','min','std']))

            sum       mean    max    min        std
count                                              
0.0   -110101.0 -15.105090  594.0 -775.0  57.010653
1.0       490.0   0.123550  453.0 -501.0  39.515494
2.0      1357.0   0.744377  521.0 -465.0  38.826480
3.0      -674.0  -0.881046   68.0 -215.0  31.945064
4.0      -484.0  -1.629630  149.0 -322.0  35.863257
5.0      -548.0  -4.528926   54.0 -236.0  37.895267
6.0      -134.0  -2.436364   36.0 -228.0  38.421523
7.0        16.0   0.800000  116.0 -131.0  47.672788
8.0        46.0  11.500000   22.0   -3.0  12.124356
9.0       -73.0 -36.500000  -25.0  -48.0  16.263456
10.0      -29.0 -29.000000  -29.0  -29.0        NaN
11.0      -13.0 -13.000000  -13.0  -13.0        NaN


In [ ]:
#输出统计结果

In [23]:
result1=total_ret1.groupby('count')['macd_section_pnl'].describe()

In [25]:
result1.to_excel('金叉统计描述.xlsx')

In [26]:
result2=total_ret1.groupby('count')['macd_section_pnl'].aggregate(['sum','mean','max','min','std'])

In [27]:
result2.to_excel('金叉统计结果.xlsx')

In [34]:
result3=total_ret2.groupby('count')['macd_section_pnl'].describe()

In [35]:
result3.to_excel('死叉统计描述.xlsx')

In [36]:
result4=total_ret2.groupby('count')['macd_section_pnl'].aggregate(['sum','mean','max','min','std'])

In [37]:
result4.to_excel('死叉统计结果.xlsx')